# Script para etiquetar las imagenes de entrenamiento v2
## Entorno de ejecución: Colab
## Autor: Juan M. Mendoza

In [1]:
# Descarga el dataset
!gdown "https://drive.google.com/uc?export=download&id=12KR_l0TgLhwc3J25GFUkd9S73OEtphG_"
!gdown "https://drive.google.com/uc?export=download&id=12ELJm48gudZApmxozqqeg4JL5HInDauU"

Downloading...
From: https://drive.google.com/uc?export=download&id=12KR_l0TgLhwc3J25GFUkd9S73OEtphG_
To: /content/images_ts.rar
147MB [00:02, 68.8MB/s]
Downloading...
From: https://drive.google.com/uc?export=download&id=12ELJm48gudZApmxozqqeg4JL5HInDauU
To: /content/images_tr.rar
342MB [00:03, 104MB/s]


In [2]:
# instala el compresor
!apt-get install rar

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-440
Use 'apt autoremove' to remove it.
The following NEW packages will be installed:
  rar
0 upgraded, 1 newly installed, 0 to remove and 35 not upgraded.
Need to get 295 kB of archives.
After this operation, 799 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/multiverse amd64 rar amd64 2:5.5.0-1 [295 kB]
Fetched 295 kB in 0s (3,529 kB/s)
Selecting previously unselected package rar.
(Reading database ... 144465 files and directories currently installed.)
Preparing to unpack .../rar_2%3a5.5.0-1_amd64.deb ...
Unpacking rar (2:5.5.0-1) ...
Setting up rar (2:5.5.0-1) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...


In [3]:
# extrae los archivos
! rar x 'images_tr.rar'


RAR 5.50   Copyright (c) 1993-2017 Alexander Roshal   11 Aug 2017
Trial version             Type 'rar -?' for help


Extracting from images_tr.rar

Creating    images_tr                                                 OK
Creating    images_tr/56                                              OK
Extracting  images_tr/56/spleen_56.ni_z026.png                             0%  OK 
Extracting  images_tr/56/spleen_56.ni_z014.png                             0%  OK 
Extracting  images_tr/56/spleen_56.ni_z008.png                             0%  OK 
Extracting  images_tr/56/spleen_56.ni_z031.png                             0%  OK 
Extracting  images_tr/56/spleen_56.ni_z030.png                             0%  OK 
Extracting  images_tr/56/spleen_56.ni_z032.png                             0%  OK 
Extracting  images_tr/56/spleen_56.ni_z001.png                             0%  OK 
Extracting  images_tr/56/spleen_56.ni_z019.png               

In [4]:
# importa las librerias que se usaran en el etiquetado
import numpy as np
import cv2 as cv
from matplotlib import pyplot as plt

In [5]:
j= [[[2 ,4]],[[3 ,5]],[[1, 6]]]
vector=np.array(j)
num = 0
sumx = 0
sumy = 0
for [[x,y]] in j:
  num = num + 1
  sumx = sumx + x
  sumy = sumy + y
print(vector)
print("- promedio (media) de los elementos:")
print([sumx/num,sumy/num])

[[[2 4]]

 [[3 5]]

 [[1 6]]]
- promedio (media) de los elementos:
[2.0, 5.0]


In [6]:
import os, sys
import glob
import pandas as pd

dirs = os.listdir('images_tr')
data = []
for file in dirs:
  #obtiene los paths para entrenamiento
  paths_tr = glob.glob("images_tr/"+file+"/*.png")
  os.makedirs("images_label_tr/"+file)
  for path in paths_tr:
    # lee la imagen y la pasa a escala de grises
    img = cv.imread(path)
    print(path)
    gray = cv.cvtColor(img,cv.COLOR_BGR2GRAY)
    ret, thresh = cv.threshold(gray,0,255,cv.THRESH_BINARY_INV+cv.THRESH_OTSU)
    # Filtrado de la imagen 
    # eliminacion de ruido
    kernel = np.ones((3,3),np.uint8)
    opening = cv.morphologyEx(thresh,cv.MORPH_OPEN,kernel, iterations = 2)
    # area de fondo
    sure_bg = cv.dilate(opening,kernel,iterations=3)
    # area de interes
    dist_transform = cv.distanceTransform(opening,cv.DIST_L2,5)
    ret, sure_fg = cv.threshold(dist_transform,0.7*dist_transform.max(),255,0)
    # encontrando la region desconocida
    sure_fg = np.uint8(sure_fg)
    unknown = cv.subtract(sure_bg,sure_fg)
    # https://stackoverflow.com/questions/22240746/recognize-open-and-closed-shapes-opencv
    contours, hierarchy = cv.findContours(unknown, cv.RETR_CCOMP, cv.CHAIN_APPROX_SIMPLE)
    hierarchy = hierarchy[0]
    num_c= 0
    x1 = 0
    x2 = 0
    y1 = 0
    y2 = 0
    for i, c in enumerate(contours):
        if hierarchy[i][2] < 0 and hierarchy[i][3] < 0:
            cv.drawContours(img, contours, i, (0, 0, 255), 2)
            
            num_c = num_c +1
            num = 0
            sumx = 0
            sumy = 0
            for [[x,y]] in c:
              num = num + 1
              sumx = sumx + x
              sumy = sumy + y
            if num_c == 1:
              x1 = int(sumx/num)
              y1 = int(sumy/num)
            if num_c == 2:
              x2 = int(sumx/num)
              y2 = int(sumy/num)
        else:
            cv.drawContours(img, contours, i, (0, 255, 0), 2)
    final_path = "images_label_tr/"+file+"/label_"+path.split('/')[2]
    print([final_path,num_c,x1,y1,x2,y2])
    data.append([final_path,num_c,x1,y1,x2,y2])
    # ploteo de region desconocida
    #plt.axis("off")
    #plt.imshow(cv.cvtColor(img, cv.COLOR_BGR2RGB))
    #plt.show()
    cv.imwrite( "images_label_tr/"+file+"/label_"+path.split('/')[2], img);
df = pd.DataFrame(data, columns=['path','num','x1','y1','x2','y2'])

Se han truncado las últimas 5000 líneas del flujo de salida.
['images_label_tr/27/label_spleen_27.ni_z016.png', 3, 298, 276, 225, 260]
images_tr/27/spleen_27.ni_z053.png
['images_label_tr/27/label_spleen_27.ni_z053.png', 3, 332, 208, 294, 136]
images_tr/27/spleen_27.ni_z042.png
['images_label_tr/27/label_spleen_27.ni_z042.png', 5, 310, 367, 319, 309]
images_tr/27/spleen_27.ni_z110.png
['images_label_tr/27/label_spleen_27.ni_z110.png', 1, 247, 161, 0, 0]
images_tr/27/spleen_27.ni_z077.png
['images_label_tr/27/label_spleen_27.ni_z077.png', 2, 348, 316, 337, 146]
images_tr/27/spleen_27.ni_z040.png
['images_label_tr/27/label_spleen_27.ni_z040.png', 4, 317, 319, 313, 217]
images_tr/27/spleen_27.ni_z044.png
['images_label_tr/27/label_spleen_27.ni_z044.png', 6, 297, 315, 319, 340]
images_tr/27/spleen_27.ni_z062.png
['images_label_tr/27/label_spleen_27.ni_z062.png', 2, 285, 379, 322, 155]
images_tr/27/spleen_27.ni_z113.png
['images_label_tr/27/label_spleen_27.ni_z113.png', 1, 250, 155, 0, 0]
i

In [7]:
# comprime los archivos etiquetados
!rar a "images_label_tr" "images_label_tr"


RAR 5.50   Copyright (c) 1993-2017 Alexander Roshal   11 Aug 2017
Trial version             Type 'rar -?' for help

Evaluation copy. Please register.

Creating archive images_label_tr.rar

Adding    images_label_tr/63/label_spleen_63.ni_z058.png                   0%  OK 
Adding    images_label_tr/63/label_spleen_63.ni_z060.png                   0%  OK 
Adding    images_label_tr/63/label_spleen_63.ni_z020.png                   0%  OK 
Adding    images_label_tr/63/label_spleen_63.ni_z046.png                   0%  OK 
Adding    images_label_tr/63/label_spleen_63.ni_z033.png                   0%  OK 
Adding    images_label_tr/63/label_spleen_63.ni_z050.png                   0%  OK 
Adding    images_label_tr/63/label_spleen_63.ni_z030.png                   0%  OK 
Adding    images_label_tr/63/label_spleen_63.ni_z027.png                   0%  OK 
Adding    images_label_tr/63/label_spleen_63.ni_z005.png                 

In [8]:
df

,path,num,x1,y1,x2,y2
0,images_label_tr/63/label_spleen_63.ni_z008.png,3,318,378,340,326
1,images_label_tr/63/label_spleen_63.ni_z057.png,3,156,286,398,320
2,images_label_tr/63/label_spleen_63.ni_z001.png,4,277,381,326,354
3,images_label_tr/63/label_spleen_63.ni_z040.png,4,341,171,338,144
4,images_label_tr/63/label_spleen_63.ni_z002.png,3,325,346,337,307
...,...,...,...,...,...,...
3645,images_label_tr/25/label_spleen_25.ni_z033.png,5,337,330,295,338
3646,images_label_tr/25/label_spleen_25.ni_z047.png,4,170,385,164,325
3647,images_label_tr/25/label_spleen_25.ni_z057.png,4,192,368,337,184
3648,images_label_tr/25/label_spleen_25.ni_z086.png,0,0,0,0,0


In [9]:
# guarda el dataframe
df.to_csv('info.csv')